### Importing relevant packages

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from nltk.corpus import stopwords
import eli5
import warnings
warnings.filterwarnings('ignore')
import joblib
import os

### Loading our data

In [2]:
path=os.getcwd()
parent=os.path.dirname(path)
data_path=os.path.join(parent,'Data fetching from api','data.csv')

In [3]:
df=pd.read_csv(data_path)
df.head()

,sentences,dialect
0,لكن بالنهاية .. ينتفض .. يغير .,IQ
1,يعني هذا محسوب على البشر .. حيونه ووحشيه .. وت...,IQ
2,مبين من كلامه خليجي,IQ
3,يسلملي مرورك وروحك الحلوه💐,IQ
4,وين هل الغيبه اخ محمد 🌸🌺,IQ


In [25]:
df.dialect.value_counts()

EG    57636
PL    43742
KW    42109
LY    36499
QA    31069
JO    27921
LB    27617
SA    26832
AE    26296
BH    26292
OM    19116
SY    16242
DZ    16183
IQ    15497
SD    14434
MA    11539
YE     9927
TN     9246
Name: dialect, dtype: int64

### Shuffling data

In [3]:
df_shuffled=df.sample(frac=1,random_state=42)

### Train/Test split

In [4]:
df_train= df_shuffled.iloc[:int(0.8*len(df_shuffled))]
df_test= df_shuffled.iloc[int(0.8*len(df_shuffled)):]


### Defining report function

In [5]:
def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

### Creating stop words list

In [6]:
stop_words_list= stopwords.words('arabic')

### Random forest

In [7]:
vec = CountVectorizer(stop_words=stop_words_list)
clf=RandomForestClassifier(n_estimators=100,max_depth=2)
pipe = make_pipeline(vec, clf)


In [8]:
pipe.fit(df_train.sentences, df_train.dialect)

Pipeline(steps=[('countvectorizer',
                 CountVectorizer(stop_words=['إذ', 'إذا', 'إذما', 'إذن', 'أف',
                                             'أقل', 'أكثر', 'ألا', 'إلا',
                                             'التي', 'الذي', 'الذين', 'اللاتي',
                                             'اللائي', 'اللتان', 'اللتيا',
                                             'اللتين', 'اللذان', 'اللذين',
                                             'اللواتي', 'إلى', 'إليك', 'إليكم',
                                             'إليكما', 'إليكن', 'أم', 'أما',
                                             'أما', 'إما', 'أن', ...])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=2))])

In [9]:
print_report(pipe, df_test.sentences, df_test.dialect)

              precision    recall  f1-score   support

          AE       0.00      0.00      0.00      5264
          BH       0.00      0.00      0.00      5113
          DZ       0.00      0.00      0.00      3142
          EG       0.13      1.00      0.23     11621
          IQ       0.00      0.00      0.00      3183
          JO       0.00      0.00      0.00      5488
          KW       0.00      0.00      0.00      8423
          LB       0.00      0.00      0.00      5517
          LY       0.00      0.00      0.00      7325
          MA       0.00      0.00      0.00      2281
          OM       0.00      0.00      0.00      3809
          PL       0.00      0.00      0.00      8814
          QA       0.00      0.00      0.00      6216
          SA       0.00      0.00      0.00      5430
          SD       0.00      0.00      0.00      2910
          SY       0.00      0.00      0.00      3217
          TN       0.00      0.00      0.00      1936
          YE       0.00    

### Logistic regression

In [10]:
vec = CountVectorizer()
clf = LogisticRegression()
pipe = make_pipeline(vec, clf)


In [11]:
pipe.fit(df_train.sentences, df_train.dialect)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('logisticregression', LogisticRegression())])

In [12]:
print_report(pipe, df_test.sentences, df_test.dialect)

              precision    recall  f1-score   support

          AE       0.41      0.42      0.41      5264
          BH       0.40      0.31      0.35      5113
          DZ       0.59      0.53      0.56      3142
          EG       0.69      0.83      0.76     11621
          IQ       0.63      0.52      0.57      3183
          JO       0.42      0.36      0.39      5488
          KW       0.47      0.56      0.51      8423
          LB       0.60      0.67      0.63      5517
          LY       0.63      0.68      0.66      7325
          MA       0.74      0.56      0.64      2281
          OM       0.41      0.34      0.37      3809
          PL       0.50      0.52      0.51      8814
          QA       0.44      0.49      0.47      6216
          SA       0.40      0.43      0.42      5430
          SD       0.70      0.55      0.62      2910
          SY       0.45      0.34      0.39      3217
          TN       0.67      0.43      0.52      1936
          YE       0.41    

### logistic regression is better than random forest

### Removing stop words doesn't increase accuracy

### Let's better understand our model

In [13]:
eli5.show_weights(clf, vec=vec, top=20)

In [14]:
pipe.predict(['مشان الله لا تقوصنى معلم'])

array(['SY'], dtype=object)

### Saving our model

In [15]:
joblib.dump(pipe, 'model.pkl')

['model.pkl']